In [2]:
import numpy as np 
import pandas as pd
import os

In [3]:
import tensorflow
from tensorflow.keras.layers import Resizing, Rescaling
from tensorflow.data import Dataset

In [5]:
data_path = "Data"

labels = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

data_directory = tensorflow.keras.utils.image_dataset_from_directory(
    data_path,
    class_names = labels,
    color_mode = 'grayscale',
    image_size =(200, 200),
)


data_directory = data_directory.map(lambda x, y: (x / 255.0, y))

Found 13008 files belonging to 26 classes.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(8, (3,3), 1, activation ='relu', input_shape=(200, 200, 1)))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation ='relu' ))
model.add(MaxPooling2D())

model.add(Conv2D(8, (3,3), 1, activation ='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dense(26, activation = 'softmax'))

In [8]:
from tensorflow.keras.optimizers import Adam, SGD

opt = Adam(learning_rate = 0.001)
# opt = SGD(learning_rate = 0.001, momentum = 0.9)

model.compile(optimizer = opt, loss ='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary() 

loss, accuracy = model.evaluate(test)

print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

final_val_loss = val_loss[-1]
final_val_accuracy = val_accuracy[-1]
print('Final Validation Loss:', final_val_loss)
print('Final Validation Accuracy:', final_val_accuracy)

epochs = range(1, num_epochs + 1)
plt.plot(epochs, val_loss, label='Validation Loss')
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()
plt.show()
